In [324]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import urllib.request
import csv
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import pandas as pd
from sklearn.utils import shuffle
from pandas_datareader import data as web
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from nltk.corpus import stopwords
import nltk
import string
from tqdm import tqdm
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chenwi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chenwi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\chenwi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [325]:
def GetStockPrice(stock, startDate, endDate, method):
    if method == 0:
        Stockdf = web.DataReader(stock, 'yahoo', startDate, endDate)
        AdjClosedf = pd.DataFrame(Stockdf['Adj Close'])
        AdjClosedf.loc[:,'Label'] = (AdjClosedf['Adj Close'].pct_change()>0).astype(int)

    else:
        page = urllib.request.urlopen('https://finance.yahoo.com/quote/'+stock+'/history?period1=1474095600&period2=1504249200&interval=1d&filter=history&frequency=1d').read()
        soup = BeautifulSoup(page, 'html5lib')
        snap = soup.find("table", class_="W(100%) M(0)")
        snap_body = snap.find('tbody')
        rows = snap_body.find_all('tr')

        #print "Number of data: %d\n" % (len(rows))
        Datelist = []
        AdjCloselist = []

        for element in rows:
            s = element.find_all("span")
            if len(s) == 7:
                Datelist.append(datetime.strptime(s[0].get_text(), '%b %d, %Y'))
                AdjCloselist.append(locale.atof(s[5].get_text().replace(',','')))


        AdjClosedf = pd.DataFrame(AdjCloselist,columns=[stock],index = Datelist)
        #print(AdjClosedf.columns)
        AdjClosedf = AdjClosedf.sort_index()
        AdjClosedf.rename(columns={stock: 'Adj Close'}, inplace=True)
        AdjClosedf.loc[:,'Label'] = (AdjClosedf['Adj Close'].pct_change()>0).astype(int)
        
    return AdjClosedf

In [326]:
def GetStockNews(stock, TopN, DateRange):
    data = {}
    for ndate in DateRange:
        data[ndate] = []
        #print ndate.strftime('%Y-%m-%d')
        startdate = ndate.strftime('%Y-%m-%d')
        enddate = startdate
        page = urllib.request.urlopen('https://finance.google.com/finance/company_news?q=NASDAQ%3A'+stock+
                                      '&startdate='+startdate+'&enddate='+enddate).read()
        soup= BeautifulSoup(page, 'html5lib')
        news = soup.find_all("div", class_="g-section news sfe-break-bottom-16")
        #print ("Number of news: %d\n" % (len(news)))
        for element in news[:TopN]:
            t = element.find_all("span", class_="name")[0].get_text().replace('\xa0',' ').replace('\n','')#.encode('utf-8')
            #print t 
            data[ndate].append(t)
        for i in range(0,TopN - len(news),1):
            data[ndate].append('')
    df = pd.DataFrame().from_dict(data).T
    return df

In [327]:
start = '2015-01-01'
end = '2017-09-01'
constituentdf = pd.read_csv('constituents.csv',)
stocklist = constituentdf.Symbol.values

Simple sentiment analysis

In [328]:
positive = pd.read_csv('positive-words.txt', names=['a'], encoding='latin-1')
positive =  set(positive['a'].tolist())

negative = pd.read_csv('negative-words.txt', names=['a'], encoding='latin-1')
negative =  set(negative['a'].tolist())

In [329]:
def addSentiment(newsInput): 
    commonp = set(newsInput.split()).intersection(positive) 
    count_positive=len(commonp)
    commonn = set(newsInput.split()).intersection(negative) 
    count_negative=len(commonn)
    return count_positive, count_negative

In [331]:
def FilterSentence(headline):
    sentence = nltk.word_tokenize(headline)
    sent = nltk.pos_tag(sentence)
    tempfiltered_words = [s[0] for s in sent if s[1][0] in ['J','V','N']]
    stops = set(stopwords.words("english"))
    exclude = set(string.punctuation+'‘’“”0123456789')
    filtered_words = ' '.join(word.lower() for word in tempfiltered_words if (word not in stops) and (word not in exclude))
    return filtered_words

In [332]:
def TrainingProcess(train):
    filtered_words = []
    for row in range(0,len(train.index)):
        headline = ' '.join(str(x).replace('\uFFFD','') for x in train.iloc[row,1:])
        trainheadlines = FilterSentence(headline)
        count_positive, count_negative=addSentiment(trainheadlines)
        for i in range(count_positive):
            trainheadlines+=' positive'
        for j in range(count_negative):
            trainheadlines+=' negative'
        filtered_words.append(trainheadlines)
    
    text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', LogisticRegression())])
    text_clf.fit(filtered_words, train["Label"])
    
    return text_clf

In [333]:
def TestProcess(test,text_clf):
    filtered_words = []
    for row in range(0,len(test.index)):
        headline = ' '.join(str(x).replace('\uFFFD','') for x in test.iloc[row,1:])
        testheadlines = FilterSentence(headline)
        count_positive, count_negative=addSentiment(testheadlines)
        for i in range(count_positive):
            testheadlines+=' positive'
        for j in range(count_negative):
            testheadlines+=' negative'
        filtered_words.append(testheadlines)

    predictions = text_clf.predict(filtered_words)
    
    return predictions

In [334]:
def calScores(y_test, y_test_pred):
    print("Accuracy score:")
    acc_score = accuracy_score(y_true=y_test, y_pred=y_test_pred)
    print("On test set: %.3f" % acc_score)
    print("\nF1 score:")
    f_score = f1_score(y_true=y_test, y_pred=y_test_pred, average='weighted')                                            
    print("On test set: %.3f" % f_score)
    crossdf = pd.crosstab(y_test, y_test_pred, rownames=["Actual"], colnames=["Predicted"])
    print('\n')
    print(crossdf)

In [335]:

TopN = 10
TrainSize = 0.8

In [336]:
#datadf = pd.DataFrame()
#inputdf = pd.read_csv(stock+'.csv',index_col=0,encoding='utf-8')
#datadf = pd.concat([datadf,inputdf],axis = 0)         


In [337]:
datadf = pd.DataFrame()
for stock in stocklist:
    if os.path.isfile('.\Data\\'+stock+'.csv'):
        #print(stock)
        inputdf = pd.read_csv('.\Data\\'+stock+'.csv',index_col=0,encoding='utf_8')
        datadf = pd.concat([datadf,inputdf],axis = 0)
print(datadf.head())

            Label                                                  0  \
2017-04-17      0  Smart Investors Are Already Buying These Stock...   
2017-04-18      0  Ann Marie Hanrahan, 3M attorney who championed...   
2017-04-19      0  More lucky locals set to share �3m windfall as...   
2017-04-20      1  A Retirement Portfolio With Quality Over Quantity   
2017-04-21      1  Epson Makes Two Announcements at ISA Expo Enha...   

                                                            1  \
2017-04-17  Two Brits arrested in Alicante on �3m drugs tr...   
2017-04-18  Global Grinding Machine Market, 2023: Leading ...   
2017-04-19  New �3m partnership to help children stay safe...   
2017-04-20  Revealed: Kirkcaldy streets set for �3m lotter...   
2017-04-21  Ex-Gonzalez Saggio Lawyers in NJ Join NY's Sch...   

                                                            2  \
2017-04-17      DWTC attracts more than 3m visitors last year   
2017-04-18  Driving Innovation through Respons

In [338]:
datadf = shuffle(datadf)
#print(datadf.shape)
datadf=datadf.dropna(axis=0, how='any')
#print(datadf.shape)
dataset_x=datadf[['0','1','2','3','4','5','6','7','8','9']]
dataset_y=datadf['Label']
print(dataset_y.shape)
trainheadlines = []
for row in range(0,len(dataset_x.index)):
    trainheadlines.append(' '.join(str(x) for x in dataset_x.iloc[row,:]))
print(trainheadlines[0])
basictrain = CountVectorizer().fit_transform(trainheadlines)
print(basictrain.shape)

(5773,)
Real estate briefly: Digital Map Products shifting HQ to University Research Park JPMorgan Chase & Co. Raises Stake in PICO Holdings Inc. (NASDAQ:PICO) iShares iBoxx $ High Yid Corp Bond (NYSE:HYG) Shares Bought by Sei Investments Co. Alio Gold Inc (NYSE:ALO) Upgraded at BMO Capital Markets PGGM Investments Has $68452000 Stake in Camden Property Trust (CPT) Temperance Flat Dam investment will pay off for California The Charles Schwab Corporation - Receive News & Ratings Daily Digipath Inc. (OTCMKTS:DIGP) Lowered to Sell at ValuEngine Week in review: Health insurers to get rate hikes; developer sentenced to 4 months Hey, Marijuana Stock Investors: This Senator Just Introduced a Bill That ...
(5773, 37014)


In [339]:

#from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import cross_validate, train_test_split

np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(basictrain, dataset_y, test_size=0.2, stratify=dataset_y)
clf = LogisticRegression(C=1.0, penalty='l1')
scoring = {'accuracy': make_scorer(accuracy_score),'prec': 'precision'}
cv_results = cross_validate(clf.fit(X_train, y_train), X_train, y_train, scoring=scoring)

"\nclf = LogisticRegression(C=1.0, penalty='l1')\nscoring = {'accuracy': make_scorer(accuracy_score),'prec': 'precision'}\ncv_results = cross_validate(clf.fit(X_train, y_train), X_train, y_train, scoring=scoring)\n"

In [340]:
for s, r in cv_results.items():
    print (s, r)

neg = y_train[y_train == -1].count()
pos = y_train[y_train == 1].count()
print (neg*100/pos)

neg2 = y_test[y_test == -1].count()
pos2 = y_test[y_test == 1].count()
print (neg2*100/pos2)


fit_time [ 0.12803125  0.1005528   0.14609575]
score_time [ 0.00400043  0.00200558  0.00399971]
test_accuracy [ 0.50584416  0.49675325  0.50390117]
train_accuracy [ 0.98830409  0.99220273  0.98928571]
test_prec [ 0.5309842   0.52247874  0.52941176]
train_prec [ 0.98651134  0.992       0.98594991]
0.0
0.0


In [341]:
traindf = datadf.iloc[:int(TrainSize*len(datadf)),:]
testdf = datadf.iloc[int(TrainSize*len(datadf)):,:]
clf = TrainingProcess(traindf)
predictions = TestProcess(testdf,clf)

calScores(testdf['Label'].values, predictions)

Accuracy score:
On test set: 0.548

F1 score:
On test set: 0.535


Predicted    0    1
Actual             
0          205  346
1          176  428
